In [ ]:
#downloading the file
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

In [ ]:
#Unziping the file
!unzip datasource.zip -d dealership_data

In [ ]:
#Set path 
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

In [ ]:
#CSV extract function below
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe


In [ ]:
#JSON extract function below
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process, lines=True)
    return dataframe

In [ ]:
# Add the XML extract function 
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["car_model", "year_of_manufacture", "price", "fuel"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index = True)
    return dataframe

In [ ]:
# Defining extract function
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

In [ ]:
# transform function
def transform(data):
    data['price'] = round(data.price, 2)
    return data

In [ ]:
#load function
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

In [ ]:
#log function 
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

In [ ]:
# Log ETL process
log("ETL Job Started")

# Log Extract step
log("ETL Extract Started")
# Call the Extract function
extracted_data = extract()
# Log completed Extract step
log("Extract Job Completed")


# Log Transform step
log("Transform Job Started")
# Call the Transform function
transformed_data = transform(extracted_data)
# Log completed the Transform step
log("Transform Job Completed") 

# Log the Load step
log("Load Job Started")
# Call the Load function
load(targetfile, transformed_data)
# Log completed the Load step
log("Load Job Completed")

# Log completed the ETL process
log("ETL Job Ended")